In [14]:
import sys
'''
notes: from my understanding of the problem, only output a row if there is a purchase. initiate all accounts as 'no history'. fraud reports don't expire and continue to count 'fraud history' regardless of purchase history. (realistically, there should be some type of fraud forgiveness plan when fraud expires but for the sake of the problem, sure). to save space, isolate a list of datetimes within 90 days as a cache and only count up for 'good history' once the datatimes expire 90 days later and delete from recent list; otherwise, it is 'unconfirmed history' regardless of how many datetimes are in cache to save some CPU. realistically in the business sense of the problem, given that fraud can be submitted within a 90 day window, the calculations have to be redone for the current version of the analysis because the data can change status posthoc, so the redundant recalculations would be advantageous
'''
from datetime import datetime, date, timedelta
customer_dict = {}

class Event:
    def __init__(self,customerAccount):
        self.customerAccountID = customerAccount
        self.historyAll = []
        self.historyPurchase = []
        self.purchaseCount = 0
        self.fraudCount = 0
        
    def getStatus(self):
        if (len(self.historyAll) == 1) and (len(self.historyPurchase) == 1):
            return str(self.historyPurchase[-1].date()) + ',' + self.customerAccountID + ',NO_HISTORY'
        elif self.fraudCount == 0:
            #this is the redundant calculation way to do it if the data has an 90 day update window of turning purchased into fraud in the event types posthoc
            unconfirmedHistory = 0
            goodHistory = 0
            currentEvent = self.historyPurchase[-1]
            for i in range(len(self.historyPurchase)-1):
                pastDate = self.historyPurchase[i]
                if (currentEvent - pastDate).days > 90:
                    goodHistory += 1
                else:
                    unconfirmedHistory += 1
            if goodHistory > 0:
                return str(self.historyPurchase[-1].date()) + ',' + self.customerAccountID + ',GOOD_HISTORY:' + str(goodHistory)
            else:
                return str(self.historyPurchase[-1].date()) + ',' + self.customerAccountID + ',UNCONFIRMED_HISTORY:' + str(unconfirmedHistory)
        elif (self.fraudCount > 0 and len(self.historyPurchase) > 0):
            return str(self.historyPurchase[-1].date()) + ',' + self.customerAccountID + ',FRAUD_HISTORY:' + str(self.fraudCount)

    def purchaseEvent(self, date, eventType):
        self.historyAll.append(date)
        if eventType == 'PURCHASE':
            self.historyPurchase.append(date)
            self.purchaseCount += 1
        else:
            self.fraudCount += 1

def parse_line(line):
    parsed_line = line.split(',')
    parsed_line[0] = datetime.strptime(parsed_line[0],'%Y-%m-%d')
    return parsed_line

def getAccountHistory(lines):
    reportLines = []
    for line in lines:
        if line[1] not in customer_dict:
            customer_dict[line[1]] = Event(line[1])
        customer_dict[line[1]].purchaseEvent(line[0],line[2])
        reportLines += filter(None, [customer_dict[line[1]].getStatus()])
    return reportLines 

CSV_lines = [
"2021-01-01,joe@signifyd.com,PURCHASE",
"2021-02-01,fraudster@fraud.com,FRAUD_REPORT",
"2021-02-03,fraudster@fraud.com,FRAUD_REPORT",
"2021-02-10,joe@signifyd.com,PURCHASE",
"2021-02-14,fraudster@fraud.com,PURCHASE",
"2021-03-15,joe@signifyd.com,PURCHASE",
"2021-05-01,joe@signifyd.com,PURCHASE",
"2021-10-01,joe@signifyd.com,PURCHASE",]

if __name__ == "__main__":
    lines_parsed: [list[list[str]]] = []
    for line in CSV_lines:
        parsed_line = line.strip('\n').split(',')
        parsed_line[0] = datetime.strptime(parsed_line[0], "%Y-%m-%d")
        lines_parsed.append(parsed_line)
        
    result = getAccountHistory(lines_parsed)
    for i in range(len(result)):
        print(result[i])

2021-01-01,joe@signifyd.com,NO_HISTORY
2021-02-10,joe@signifyd.com,UNCONFIRMED_HISTORY:1
2021-02-14,fraudster@fraud.com,FRAUD_HISTORY:2
2021-03-15,joe@signifyd.com,UNCONFIRMED_HISTORY:2
2021-05-01,joe@signifyd.com,GOOD_HISTORY:1
2021-10-01,joe@signifyd.com,GOOD_HISTORY:4
